#검증세트
이전에는 train 세트와 test 세트로 모델을 훈련시키고 평가했다.<br>
하지만 test세트로 평가하고 test세트에 부적합 했을 때, 다시 test세트를의 score를 높이려고 한다.<br>
이 말은 모델을 train 세트로 학습하고 test 세트로 맞추는 것이라고 할 수 있다.<br>
이런 문제를 해결 하기 위해서 기존 train, test set으로 split 하는 것에 ***검증 세트***를 추가해서 데이터를 나눌 것이다.

In [97]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [98]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [99]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, random_state=42, test_size=0.2)

In [100]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, random_state=42, test_size=0.2)

In [101]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [102]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


#교차 검증
많은 데이터로 훈련 시킬 수록 좋은 모델이 된다. 그렇다고 검증 세트를 너무 작게 떼어 내면 검증이 제대로 이뤄지지 않는다. 이럴 때, ***교차검증***을 이용하면 안정적인 검증 점수와 많은 데이터를 훈련에 사용할 수 있다.<BR>
sklearn에 cross_validate()라는 교차 검증 함수가 있다.<br>
사용 방법은 첫번째, 매개변수에 평가할 모델 객체, train_input, train_target
---
교차 검증은 검증 세트를 떼어 내는 평가하는 과정을 여러 번 반복한다. 그 다음 점수를 **평균**하여 최종 검증 점수를 얻는다. 일반적으로 5-폴드, 10-폴드<br>
ex) 3-fold coss validation<br>
훈련세트, 훈련세트, 검증세트<br>
훈련세트, 검증세트, 훈련세트<br>
검증세트, 훈련세트, 훈련세트<br>



In [103]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)
import numpy as np
print(np.mean(scores['test_score']))  # 교차 검증의 최종 점수

{'fit_time': array([0.01149201, 0.00732565, 0.00752521, 0.00747681, 0.00891709]), 'score_time': array([0.00108218, 0.00079155, 0.00085592, 0.00079322, 0.00091147]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


### cross_validate
output<br>
dict
fit_time 모델을 훈련하는 시간<br>
score_time 모델을 검증하는 시간<br>
test_score 검증세트로 평가한 점수<br>

---
회귀 모델일 경우: KFold 분할기<br>
분류 모델일 경우: StratifiedKFold<br>

---
cross_validate는 훈련세트를 섞어 폴드를 나누지 않는다.(섞고 싶다면 cv = splitter 분할기 (KFold, StratifiedKFold)사용)<br>
분할기에 n_splits 매개변수에 n 값을 넣었다면 n-fold cross validation


In [104]:
# 위 코드와 동일
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [105]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
score = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.855300214703487


# ***최적의 모델을 위한 하이퍼파라미터 탐색***

#그리드 서치(GridSearch)
하이퍼파라미터 탐색과 교차 검증을 한번에 수행한다.(cross_validate() 함수를 호출할 필요 없음)<br>
sklearn.model_selection의 GridSearchCV 클래스<br>
ex) 아래 결정 트리 모델에서 min_impurity_decrease 매개 변수의 최적값을 찾는 예제

n_jobs = -1 병렬 실행에 사용할 CPU 코어 수를 지정 기본값 1, (-1: 모든 코어)<br>
GridSearchCV의 best_estimator_ 속성에 검증 점수가 가장 높은 모델이 저장되어 있음.


In [106]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [107]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [108]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [109]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [110]:
print(gs.cv_results_)
print(gs.cv_results_['mean_test_score'])

{'mean_fit_time': array([0.01005912, 0.00823369, 0.00824933, 0.00788636, 0.00666676]), 'std_fit_time': array([0.00141344, 0.00035414, 0.00065455, 0.00067027, 0.00019055]), 'mean_score_time': array([0.00205145, 0.00116706, 0.00109887, 0.00125065, 0.00092096]), 'std_score_time': array([0.00183917, 0.00010181, 0.00011662, 0.00026602, 0.00010711]), 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}], 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]), 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]), 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216, 0.88161694, 0.8825794 ]), 

In [111]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [112]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
'max_depth': range(5, 20, 1),
'min_samples_split': range(2, 100, 10)}

In [113]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [114]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [115]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


#랜덤 서치(Random Search)
랜덤 서치는 매개변수의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달

In [116]:
from scipy.stats import uniform, randint
# 모두 주어진 볌위에서 고르게 값을 뽑는 클래스, '균등분포에서 샘플링한다'라고 말한다.
# uniform은 실수 값, randint 정수 값을 뽑는다.
rgen = randint(0, 10) 
print(rgen.rvs(10))  # 10개 뽑음

[0 8 8 9 5 4 9 0 4 3]


In [117]:
np.unique(rgen.rvs(1000), return_counts=True)  # 1000개 뽑음

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([105, 109,  79, 111,  95,  97,  94, 109, 101, 100]))

In [118]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.54727713, 0.42705956, 0.72364543, 0.54309522, 0.20364394,
       0.32454559, 0.40244211, 0.07961767, 0.57805934, 0.22688986])

In [119]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)}

In [120]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f452ed9dfd0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f452f008c90>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f452ebfb250>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f452ed9d650>},
                   random_state=42)

In [121]:
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [122]:
print(np.max(rs.cv_results_['mean_test_score']))

0.8695428296438884


In [123]:
dt = rs.best_estimator_
print(dt.score(test_input, test_target))

0.86


In [139]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42, splitter='random'), params, n_iter=100, n_jobs=-1, random_state=42)  # 객체 생성
rs.fit(train_input, train_target)  # 모델 훈련
print(rs.best_params_)  # 매개변수 조합의 best 출력
dt = rs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.786923076923077
